# Next Step


Discuss general algorithm: need to approximate gradient for back propagation. Then present gradient approximation methods.
- Closed-Form Decisions
- Linear Decision Objective
- Quadratic Decision Objective
- Generic Decision Objective

Gradient Free Methods

Experiments

Methods to compare:
- Two-stage: prediction then decision, prediction then fair decision, fair prediction then decision, fair prediction then fair decision
- DFL: DFL version of each of the above two-stage settings


Performance measures to report:
- Prediction accuracy: mean square errors of $r$ and $\hat{r}$
- Decision accuracy: mean square errors of $d(r)$ and $d(\hat{r})$
- Prediction fairness: prediction fairness measure of $\hat{r}$
- Decision fairness: decision fairness measure of $d(\hat(r))$
- Runtime of algorithm

In [189]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
from sklearn.preprocessing import StandardScaler
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.insert(0, 'E:\\User\\Stevens\\Code\\The Paper\\algorithm')
from torch.utils.data import Dataset, DataLoader


import warnings
warnings.filterwarnings("ignore")

from myutil import *
from features import get_all_features

In [190]:
# Train the Prediction Model
class RiskDataset(Dataset):
    def __init__(self, features, risks):
        self.features = torch.FloatTensor(features)
        self.risks = torch.FloatTensor(risks).reshape(-1, 1)
        
    def __len__(self):
        return len(self.features)
        
    def __getitem__(self, idx):
        return self.features[idx], self.risks[idx]
    
class RiskPredictor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1),
            nn.Softplus()
        )
    
    def forward(self, x):
        return self.model(x)

# Training function
def train_model(features, risks, epochs=10, batch_size=32):
    dataset = RiskDataset(features, risks)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    model = RiskPredictor(features.shape[1])
    model.train()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    
    for epoch in range(epochs):
        for batch_features, batch_risks in dataloader:
            optimizer.zero_grad()
            predictions = model(batch_features)
            loss = criterion(predictions, batch_risks)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 5 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    
    return model

In [191]:
df = pd.read_csv('data/data.csv')

columns_to_keep = [
    'risk_score_t', 'program_enrolled_t', 'cost_t', 'cost_avoidable_t', 'race', 'dem_female', 'gagne_sum_tm1', 'gagne_sum_t', 
    'risk_score_percentile', 'screening_eligible', 'avoidable_cost_mapped', 'propensity_score', 'g_binary', 
    'g_continuous', 'utility_binary', 'utility_continuous'
]
# for race 0 is white, 1 is black
df_stat = df[columns_to_keep]
df_feature = df[[col for col in df.columns if col not in columns_to_keep]]

# Replace all values less than 0.1 with 0.1
#df['risk_score_t'] = df['risk_score_t'].apply(lambda x: 0.1 if x < 0.1 else x)
df['g_continuous'] = df['g_continuous'].apply(lambda x: 0.1 if x < 0.1 else x)

# subset a sample of 5000 rows of df
# df = df.sample(n=10000, random_state=1)

df.shape

(48784, 168)

In [192]:
# Define input variables for DFL
feats = df[get_all_features(df)].values
risk = df['risk_score_t'].values
gainF = df['g_continuous'].values
decision = df['propensity_score'].values
cost = np.ones(risk.shape)
race = df['race'].values
alpha = 0.5
Q = 1000

from sklearn.model_selection import train_test_split

# Perform train-test split
feats_train, feats_test, gainF_train, gainF_test, risk_train, risk_test, cost_train, cost_test, race_train, race_test = train_test_split(
    feats, gainF, risk, cost, df['race'].values, test_size=0.4, random_state=42
)



# Prediction Stage

In [193]:
scaler = StandardScaler()
feats = scaler.fit_transform(feats)

# model = train_model(feats, risk)
# torch.save(model.state_dict(), 'risk_predictor_model.pth')

# Load the model from local
model = RiskPredictor(feats.shape[1])
model.load_state_dict(torch.load('risk_predictor_model.pth'))
model.eval()

pred_risk = model(torch.FloatTensor(feats)).detach().numpy().flatten()

pred_risk.mean(), risk.mean()

(2.7172887, 4.393691680358348)

In [194]:
# True Risk Distribution
distribution_stats = df_stat.groupby('race')['risk_score_t'].describe()
print(distribution_stats)

# Predicted Risk Distribution
pred_risk_distribution = pd.DataFrame({'race': df['race'], 'pred_risk': pred_risk})
distribution_stats_pred_risk = pred_risk_distribution.groupby('race')['pred_risk'].describe()
print(distribution_stats_pred_risk)

        count      mean       std  min       25%       50%       75%  \
race                                                                   
0     43202.0  4.266933  5.102404  0.0  1.426873  2.870732  5.282827   
1      5582.0  5.374740  7.980310  0.0  1.494819  3.023611  6.030236   

             max  
race              
0     100.000000  
1      96.381858  
        count      mean       std  min       25%       50%       75%  \
race                                                                   
0     43202.0  2.554104  4.139664  0.0  0.053036  0.775554  3.488975   
1      5582.0  3.980262  6.118409  0.0  0.111057  1.603591  5.302370   

            max  
race             
0     51.777321  
1     60.639240  


# Train a Fair Regression Model

### Specifically, we'll minimize the difference in mean predictions between the two racial groups (statistical parity). The total loss will be a combination of the Mean Squared Error and the fairness regularizer.

In [195]:
# Add 'race' to the dataset
class FairRiskDataset(Dataset):
    def __init__(self, features, races, risks):
        self.features = torch.FloatTensor(features)
        self.races = torch.LongTensor(races)
        self.risks = torch.FloatTensor(risks).reshape(-1, 1)
        
    def __len__(self):
        return len(self.features)
        
    def __getitem__(self, idx):
        return self.features[idx], self.races[idx], self.risks[idx]

class FairRiskPredictor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1),
            nn.Softplus()
        )
        
    def forward(self, x):
        return self.model(x)


In [196]:
def train_fair_model(features, races, risks, epochs=10, batch_size=32, lambda_fairness=0.8):
    """
    Train a fair regression model with a fairness regularizer.
    
    Args:
        features (np.ndarray): Feature array.
        races (np.ndarray): Array indicating race (0: white, 1: black).
        risks (np.ndarray): True risk values.
        epochs (int): Number of training epochs.
        batch_size (int): Batch size for training.
        lambda_fairness (float): Weight for the fairness regularizer.
        
    Returns:
        nn.Module: Trained fair regression model.
    """
    dataset = FairRiskDataset(features, races, risks)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    model = FairRiskPredictor(features.shape[1])
    model.train()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        for batch_features, batch_races, batch_risks in dataloader:
            optimizer.zero_grad()
            predictions = model(batch_features)
            mse_loss = criterion(predictions, batch_risks)
            
            # Compute fairness loss
            group0 = predictions[batch_races == 0]
            group1 = predictions[batch_races == 1]
            if len(group0) > 0 and len(group1) > 0:
                fairness_loss = torch.abs(group0.mean() - group1.mean())
            else:
                fairness_loss = torch.tensor(0.0)
            
            # Total loss
            total_loss = mse_loss + lambda_fairness * fairness_loss
            total_loss.backward()
            optimizer.step()
            
            epoch_loss += total_loss.item()
        
        if (epoch + 1) % 5 == 0 or epoch == 0:
            avg_loss = epoch_loss / len(dataloader)
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')
    
    return model


In [197]:
# Extract necessary columns
features = df[get_all_features(df)].values
races = df_stat['race'].values  # 0: white, 1: black
risks = df_stat['risk_score_t'].values

# Drop rows with any NaNs or Infs
mask = ~np.isnan(features).any(axis=1) & ~np.isinf(features).any(axis=1) & \
       ~np.isnan(races) & ~np.isinf(races) & \
       ~np.isnan(risks) & ~np.isinf(risks)

features = features[mask]
races = races[mask]
risks = risks[mask]


# Scale features
scaler_fair = StandardScaler()
features_scaled = scaler_fair.fit_transform(features)

# Train the fair regression model
lambda_fairness = 1  # Adjust this value as needed
# fair_model = train_fair_model(features_scaled, races, risks, epochs=20, batch_size=64, lambda_fairness=lambda_fairness)

# # Save the fair model
# torch.save(fair_model.state_dict(), 'fair_risk_predictor_model.pth')

# load the model
fair_model = FairRiskPredictor(features_scaled.shape[1])
fair_model.load_state_dict(torch.load('fair_risk_predictor_model.pth'))


<All keys matched successfully>

In [198]:
fair_model, model

(FairRiskPredictor(
   (model): Sequential(
     (0): Linear(in_features=149, out_features=1, bias=True)
     (1): Softplus(beta=1, threshold=20)
   )
 ),
 RiskPredictor(
   (model): Sequential(
     (0): Linear(in_features=149, out_features=1, bias=True)
     (1): Softplus(beta=1, threshold=20)
   )
 ))

# Solve Optimization Problem

In [199]:
def AlphaFairness(util,alpha):
    if alpha == 1:
        return np.sum(np.log(util))
    elif alpha == 0:
        return np.sum(util)
    elif alpha == 'inf':
        return np.min(util)
    else:
        return np.sum(util**(1-alpha)/(1-alpha))
    
def solve_optimization(gainF, risk, cost, alpha, Q):
    # Flatten input arrays
    gainF, risk, cost = gainF.flatten(), risk.flatten() + 0.001, cost.flatten()
    d = cp.Variable(risk.shape, nonneg=True)
    
    utils = cp.multiply(cp.multiply(gainF, risk), d)
    
    if alpha == 'inf':
        # Maximin formulation
        t = cp.Variable()  # auxiliary variable for minimum utility
        objective = cp.Maximize(t)
        constraints = [
            d >= 0,
            # d <= 1,
            cp.sum(cost * d) <= Q,
            utils >= t  # t is the minimum utility
        ]
    elif alpha == 1:
        # Nash welfare (alpha = 1)
        objective = cp.Maximize(cp.sum(cp.log(utils)))
        constraints = [
            d >= 0,
            # d <= 1,
            cp.sum(cost * d) <= Q
        ]
    elif alpha == 0:
        # Utilitarian welfare (alpha = 0)
        objective = cp.Maximize(cp.sum(utils))
        constraints = [
            d >= 0,
            # d <= 1,
            cp.sum(cost * d) <= Q
        ]
    else:
        # General alpha-fairness
        objective = cp.Maximize(cp.sum(utils**(1-alpha))/(1-alpha) if alpha != 0 
                              else cp.sum(utils))
        constraints = [
            d >= 0,
            # d <= 1,
            cp.sum(cost * d) <= Q
        ]
    
    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.MOSEK, verbose=False, warm_start=True, mosek_params={'MSK_IPAR_LOG': 1})
    
    if problem.status != 'optimal':
        print(f"Warning: Problem status is {problem.status}")
    
    optimal_decision = d.value
    optimal_value = AlphaFairness(optimal_decision * gainF * risk, alpha)
    
    return optimal_decision, optimal_value

In [200]:
# pred_sol,_ = solve_optimization(gainF, predicted_risk, cost, alpha='inf', Q=Q)
# pred_obj = np.sum((risk * gainF * pred_sol)**(1-alpha)/(1-alpha))
# true_obj = np.sum((optimal_decision * gainF * risk)**(1-alpha)/(1-alpha))

In [201]:
def twoStagePTO(model, feats, gainF, risk, cost, Q, alphas=[0.5]):
    """
    Perform a two-stage optimization analysis with predictions and calculate normalized regrets.

    Args:
        model (nn.Module): A regression neural network for risk prediction.
        feats (np.ndarray): Feature array for predictions.
        gainF (np.ndarray): Gain factors.
        risk (np.ndarray): True risk values.
        cost (np.ndarray): Cost constraints.
        Q (float): Budget constraint.
        alphas (list): List of alpha values for fairness.

    Returns:
        pd.DataFrame: A table of prediction risk means, true risk mean, objectives, and normalized regrets.
    """

    # Feature scaling
    scaler = StandardScaler()
    feats_scaled = scaler.fit_transform(feats)

    # Predict risks
    model.eval()
    pred_risk = model(torch.FloatTensor(feats_scaled)).detach().numpy().flatten()

    # Initialize result storage
    results = []

    # Iterate over alphas
    for alpha in alphas:
        # Solve optimization problems
        true_sol, _ = solve_optimization(gainF, risk, cost, alpha, Q)
        pred_sol, _ = solve_optimization(gainF, pred_risk, cost, alpha, Q)

        # Calculate true and predicted utilities
        true_utility = (risk + 0.001) * gainF * true_sol
        pred_utility = (pred_risk + 0.001) * gainF * pred_sol
        pred_utility_truerisk = (risk + 0.001) * gainF * pred_sol

        # Calculate objectives
        true_obj = AlphaFairness(true_utility, alpha)
        pred_obj = AlphaFairness(pred_utility, alpha)
        pred_obj_truerisk = AlphaFairness(pred_utility_truerisk, alpha)

        # Calculate regret and normalized regret
        # regret = true_obj - pred_obj
        regret = true_obj - pred_obj_truerisk
        normalized_regret = regret / (abs(true_obj) + 1e-7)        

        # Collect results for this alpha
        results.append({
            'Alpha': alpha,
            'Predicted Risk Mean': pred_risk.mean(),
            'True Risk Mean': risk.mean(),
            'True Objective': true_obj,
            'Predicted Objective': pred_obj,
            'Regret': f"{regret:.2f}",
            'Normalized Regret': f"{normalized_regret:.2f}"
        })

    # Create a DataFrame for results
    results_df = pd.DataFrame(results)
    print(results_df)
    return results_df


In [202]:
data_sample = df.sample(n=40000, random_state=42)
feats_sample = data_sample[get_all_features(data_sample)].values
risk_sample = data_sample['risk_score_t'].values
gainF_sample = data_sample['g_continuous'].values
decision_sample = data_sample['propensity_score'].values
cost_sample = np.ones(risk_sample.shape)
race_sample = data_sample['race'].values

In [203]:
# results.to_csv('data/results.csv', index=False)
# results = twoStagePTO(model, feats_sample, gainF_sample, risk_sample, cost_sample, Q, alphas=[0,.5,.9,1,2,'inf'])
results = pd.read_csv('data/results.csv')
results

,Alpha,Predicted Risk Mean,True Risk Mean,True Objective,Predicted Objective,Regret,Normalized Regret
0,0.0,2.703654,4.387568,284164.896752,7.960235e+06,104721.50,0.37
1,0.5,2.703654,4.387568,73629.284021,3.811415e+04,23887.93,0.32
2,0.9,2.703654,4.387568,361873.257701,3.060051e+05,658.10,0.00
3,1.0,2.703654,4.387568,-44900.932086,-1.127005e+05,-0.00,-0.00
4,2.0,2.703654,4.387568,-834952.038551,-1.546416e+06,2847299.59,3.41
5,inf,2.703654,4.387568,0.000627,3.833028e-03,0.00,1.00


In [204]:
def twoStagePTO_with_bias_analysis(model, fair_model, feats, gainF, risk, cost, race, Q=1000, alphas=[0.5],):
    # Feature scaling
    scaler = StandardScaler()
    feats_scaled = scaler.fit_transform(feats)

    # Predict risks
    model.eval()
    pred_risk = model(torch.FloatTensor(feats_scaled)).detach().numpy().flatten()
    fair_pred_risk = fair_model(torch.FloatTensor(feats_scaled)).detach().numpy().flatten()

    # Initialize result storage
    results = []
    bias_analysis = []
    fair_pto_results = []
    fair_pto_analysis = []

    # Iterate over alphas
    for alpha in alphas:
        # Solve optimization problems
        true_sol, _ = solve_optimization(gainF, risk, cost, alpha, Q)
        pred_sol, _ = solve_optimization(gainF, pred_risk, cost, alpha, Q)
        fair_pred_sol, _ = solve_optimization(gainF, fair_pred_risk, cost, alpha, Q)

        # Calculate true and predicted utilities
        true_utility = (risk + 0.001) * gainF * true_sol
        pred_utility = (pred_risk + 0.001) * gainF * pred_sol
        pred_utility_truerisk = (risk + 0.001) * gainF * pred_sol
        fair_pred_utility_truerisk = (risk + 0.001) * gainF * fair_pred_sol

        # Calculate objectives
        true_obj = AlphaFairness(true_utility, alpha)
        pred_obj = AlphaFairness(pred_utility, alpha)
        pred_obj_truerisk = AlphaFairness(pred_utility_truerisk, alpha)
        fair_pred_obj_truerisk = AlphaFairness(fair_pred_utility_truerisk, alpha)

        # Calculate regret and normalized regret
        # regret = true_obj - pred_obj
        regret = true_obj - pred_obj_truerisk
        normalized_regret = regret / (abs(true_obj) + 1e-7)

        fair_regret = true_obj - fair_pred_obj_truerisk
        fair_normalized_regret = fair_regret / (abs(true_obj) + 1e-7)

        # Collect results for this alpha
        results.append({
            'Alpha': alpha,
            'Predicted Risk Mean': pred_risk.mean(),
            'True Risk Mean': risk.mean(),
            'True Objective': true_obj,
            'Predicted Objective': pred_obj,
            'Regret': f"{regret:.2f}",
            'Normalized Regret': f"{normalized_regret:.2f}"
        })

        # Add racial results
        fair_pto_results.append({
            'Alpha': alpha,
            'Predicted Risk Mean': fair_pred_risk.mean(),
            'True Risk Mean': risk.mean(),
            'True Objective': true_obj,
            'Predicted Objective': fair_pred_obj_truerisk,
            'Regret': f"{fair_regret:.2f}",
            'Normalized Regret': f"{fair_normalized_regret:.2f}"
        })

        # Analyze bias in the optimal solution and utilities by race
        for r in [0, 1]:  # 0 = white, 1 = black
            mask = race == r
            race_stats = {
                'Alpha': alpha,
                'Race': r,
                'True Solution Mean': true_sol[mask].mean(),
                'True Solution Std': true_sol[mask].std(),
                'Predicted Solution Mean': pred_sol[mask].mean(),
                'Predicted Solution Std': pred_sol[mask].std(),
                'True Utility Mean': true_utility[mask].mean(),
                'True Utility Std': true_utility[mask].std(),
                'Predicted Utility Mean': pred_utility_truerisk[mask].mean(),
                'Predicted Utility Std': pred_utility_truerisk[mask].std()
            }
            bias_analysis.append(race_stats)
        
        # Analyze bias in the optimal solution and utilities by race in for fair model
        for r in [0, 1]:
            mask = race == r
            fair_stats = {
                'Alpha': alpha,
                'Race': r,
                'True Solution Mean': true_sol[mask].mean(),
                'True Solution Std': true_sol[mask].std(),
                'Predicted Solution Mean': fair_pred_sol[mask].mean(),
                'Predicted Solution Std': fair_pred_sol[mask].std(),
                'True Utility Mean': true_utility[mask].mean(),
                'True Utility Std': true_utility[mask].std(),
                'Predicted Utility Mean': fair_pred_utility_truerisk[mask].mean(),
                'Predicted Utility Std': fair_pred_utility_truerisk[mask].std()
            }
            fair_pto_analysis.append(fair_stats)


    # Create DataFrames for results and bias analysis
    results_df = pd.DataFrame(results)
    bias_analysis_df = pd.DataFrame(bias_analysis)
    bias_analysis_df['Race'] = bias_analysis_df['Race'].replace({0: 'White', 1: 'Black'})

    fair_pto_results_df = pd.DataFrame(fair_pto_results)
    fair_pto_analysis_df = pd.DataFrame(fair_pto_analysis)
    fair_pto_analysis_df['Race'] = fair_pto_analysis_df['Race'].replace({0: 'White', 1: 'Black'})

    return results_df, bias_analysis_df, fair_pto_results_df, fair_pto_analysis_df


In [205]:
# racial_results, racial_bias_analysis, fair_pto_results, fair_pto_analysis = twoStagePTO_with_bias_analysis(model, fair_model, feats_sample, gainF_sample, risk_sample, cost_sample, race_sample, alphas=[0,.5,.9,1,2,'inf'])

# racial_bias_analysis = racial_bias_analysis.round(8)
# fair_pto_analysis = fair_pto_analysis.round(8)

# racial_bias_analysis.to_csv('data/racial_bias_analysis.csv', index=False)
# fair_pto_analysis.to_csv('data/fair_pto_analysis.csv', index=False)
# fair_pto_results.to_csv('data/fair_pto_results.csv', index=False)

In [206]:
fair_pto_results = pd.read_csv('data/fair_pto_results.csv')
racial_bias_analysis = pd.read_csv('data/racial_bias_analysis.csv')
fair_pto_analysis = pd.read_csv('data/fair_pto_analysis.csv')

fair_pto_results.groupby('Alpha').mean()

,Predicted Risk Mean,True Risk Mean,True Objective,Predicted Objective,Regret,Normalized Regret
Alpha,,,,,,
0.0,2.733022,4.387568,284164.896752,1.766419e+05,107522.97,0.38
0.5,2.733022,4.387568,73629.284021,5.448407e+04,19145.22,0.26
0.9,2.733022,4.387568,361873.257701,3.613766e+05,496.65,0.00
1.0,2.733022,4.387568,-44900.932086,-4.490093e+04,0.00,0.00
2.0,2.733022,4.387568,-834952.038551,-3.321640e+06,2486687.55,2.98
inf,2.733022,4.387568,0.000627,2.412388e-06,0.00,1.00


In [207]:
df['risk_score_t'].groupby(df['race']).mean()
df['g_continuous'].groupby(df['race']).mean()
df.race.value_counts()
df['risk_score_t'].groupby(df['race']).mean() * df['g_continuous'].groupby(df['race']).mean()
fair_pto_analysis.groupby(['Alpha', 'Race']).mean()
racial_bias_analysis.groupby(['Alpha', 'Race']).mean()

True Solution Mean  True Solution Std  Predicted Solution Mean  \
Alpha Race                                                                    
0.0   Black            0.000000           0.000000                 0.000000   
      White            0.028225           5.312689                 0.028225   
0.5   Black            0.023181           0.035950                 0.028787   
      White            0.025235           0.036278                 0.024511   
0.9   Black            0.024675           0.003974                 0.025626   
      White            0.025042           0.004007                 0.024919   
1.0   Black            0.025000           0.000010                 0.025000   
      White            0.025000           0.000010                 0.025000   
2.0   Black            0.026925           0.227790                 0.022331   
      White            0.024752           0.215799                 0.025344   
inf   Black            0.027516           0.413693                 0.026002   
      White            0.024675           0.391829                 0.024871   

             Predicted Solution Std  True Utility Mean  True Utility Std  \
Alpha Race                                                                 
0.0   Black            0.000000e+00           0.000000          0.000000   
      White            5.312689e+00           8.020686       1509.679826   
0.5   Black            1.233266e-01           2.479947          6.371667   
      White            1.855345e-01           2.646779          6.408446   
0.9   Black            4.277230e-03           0.945569          1.602100   
      White            4.393810e-03           1.029735          1.615768   
1.0   Black            2.340000e-06           0.785466          1.218131   
      White            9.700000e-07           0.855046          1.229236   
2.0   Black            6.234881e-02           0.152662          0.119106   
      White            5.960121e-02           0.162163          0.120497   
inf   Black            4.189160e-01           0.000627          0.000000   
      White            3.555798e-01           0.000627          0.000000   

             Predicted Utility Mean  Predicted Utility Std  
Alpha Race                                                  
0.0   Black                0.000000               0.000000  
      White                5.064873             953.327050  
0.5   Black                1.866960              15.644671  
      White                1.867408              32.928870  
0.9   Black                0.843973               1.361967  
      White                0.904438               1.371417  
1.0   Black                0.785445               1.218099  
      White                0.855023               1.229204  
2.0   Black                0.492565               0.810503  
      White                0.584593               0.858324  
inf   Black                0.089402               0.791130  
      White                0.104034               0.545042

# Solve in Closed Form

In [208]:
import numpy as np

def compute_d_star_closed_form(c, r, g, Q, alpha):
    """
    Compute the optimal decision variables d* using the closed-form solution,
    handling special cases for alpha = 0, 1, and 'inf'.
    
    Parameters:
    - c (np.ndarray): Array of costs (c_i), shape (n,)
    - r (np.ndarray): Array of risks (r_i), shape (n,)
    - g (np.ndarray): Array of gain factors (g_i), shape (n,)
    - Q (float): Total budget constraint
    - alpha (float or str): Fairness parameter. Use 'inf' for alpha = infinity.
    
    Returns:
    - d_star_closed (np.ndarray): Optimal decision variables (d_i*), shape (n,)
    
    Raises:
    - ValueError: If any of the inputs are invalid (e.g., non-positive costs, risks, gains, or budget).
    """
    # Input validation
    if not isinstance(c, np.ndarray) or not isinstance(r, np.ndarray) or not isinstance(g, np.ndarray):
        raise TypeError("c, r, and g must be numpy arrays.")
    if c.shape != r.shape or c.shape != g.shape:
        raise ValueError("c, r, and g must have the same shape.")
    if np.any(c < 0):
        raise ValueError("All cost values must be positive.")
    if np.any(r < 0):
        raise ValueError("All risk values must be positive.")
    if np.any(g < 0):
        raise ValueError("All gain factors must be positive.")
    if Q <= 0:
        raise ValueError("Total budget Q must be positive.")
    
    n = len(c)
    
    # Handle special cases based on alpha
    if alpha == 0:
        # Utilitarian maximization: maximize sum(r_i * g_i * d_i)
        # Optimal solution: Allocate all budget to the decision variable with the highest (r_i * g_i / c_i)
        ratios = (r * g) / c
        sorted_indices = np.argsort(-ratios)  # Descending order
        d_star_closed = np.zeros(n)
        remaining_Q = Q
        
        for idx in sorted_indices:
            if remaining_Q <= 0:
                break
            # Since there are no upper bounds on d_i, allocate as much as possible
            # Here, assuming no upper limit, allocate all remaining budget
            d_star_closed[idx] = remaining_Q / c[idx]
            remaining_Q = 0  # Budget exhausted
        
        # If any residual budget remains due to numerical issues, distribute proportionally
        if remaining_Q > 1e-6:
            allocation_ratio = (r * g) / c
            sum_ratio = np.sum(allocation_ratio)
            if sum_ratio > 0:
                d_star_closed += (allocation_ratio / sum_ratio) * remaining_Q
        
    elif alpha == 1:
        # Nash Welfare: maximize sum(log(r_i * g_i * d_i))
        # Solution: d_i* = Q / (n * c_i)
        if n == 0:
            raise ValueError("No decision variables to allocate.")
        d_star_closed = Q / (n * c)
    
    elif alpha == 'inf':
        # Maximin Fairness: maximize min(r_i * g_i * d_i)
        # Solution: Allocate d_i* proportional to 1 / (r_i * g_i)
        allocation_ratio = 1 / (r * g)
        sum_ratio = np.sum(allocation_ratio)
        if sum_ratio == 0:
            raise ValueError("Sum of 1 / (r * g) is zero, cannot allocate budget.")
        d_star_closed = (allocation_ratio / sum_ratio) * Q
    
    else:
        # General alpha-fairness for alpha > 1
        if alpha < 0:
            raise ValueError("Alpha must be non-negative.")
        
        # Compute log(numerator) to prevent numerical underflow/overflow
        log_numerator = (-1 / alpha) * np.log(c) + (-1 + 1 / alpha) * np.log(r * g)
        
        # Shift log_numerator for numerical stability
        max_log = np.max(log_numerator)
        log_numerator_shifted = log_numerator - max_log
        
        # Compute numerator in log-space
        numerator = np.exp(log_numerator_shifted)
        
        # Compute denominator using log-sum-exp for numerical stability
        sum_numerator = np.sum(numerator)
        if sum_numerator == 0:
            raise ValueError("Sum of numerators is zero, cannot allocate budget.")
        
        # Normalize to get allocation ratios and scale by Q
        d_star_closed = (numerator / sum_numerator) * Q
    
    return d_star_closed


In [209]:



def analyze_race_stats(alpha, race, true_sol, pred_sol, true_utility, pred_utility, race_labels={0: 'White', 1: 'Black'}):
    """
    Analyze and compute statistics based on race.

    Parameters:
    - alpha (float): Fairness parameter
    - race (np.ndarray): Array indicating race (e.g., 0 for White, 1 for Black)
    - true_sol (np.ndarray): True optimal solutions
    - pred_sol (np.ndarray): Predicted optimal solutions
    - true_utility (np.ndarray): True utilities
    - pred_utility (np.ndarray): Predicted utilities
    - race_labels (dict): Mapping from race codes to labels

    Returns:
    - stats (list of dict): List containing statistics per race
    """
    stats = []
    for r_code, r_label in race_labels.items():
        mask = race == r_code
        if np.sum(mask) == 0:
            continue  # Skip if no instances of this race
        race_stats = {
            'Alpha': alpha,
            'Race': r_label,
            'True Solution Mean': np.mean(true_sol[mask]),
            'True Solution Std': np.std(true_sol[mask]),
            'Predicted Solution Mean': np.mean(pred_sol[mask]),
            'Predicted Solution Std': np.std(pred_sol[mask]),
            'True Utility Mean': np.mean(true_utility[mask]),
            'True Utility Std': np.std(true_utility[mask]),
            'Predicted Utility Mean': np.mean(pred_utility[mask]),
            'Predicted Utility Std': np.std(pred_utility[mask])
        }
        stats.append(race_stats)
    return stats

def twoStagePTO_with_bias_analysis(
    model, fair_model, feats, gainF, risk, cost, race, 
    Q=1000, alphas=[0.5]
):
    """
    Perform two-stage PTO with bias analysis by computing both solver-based and closed-form solutions.

    Parameters:
    - model: PyTorch model for risk prediction
    - fair_model: PyTorch model for fair risk prediction
    - feats (np.ndarray): Feature matrix, shape (n_samples, n_features)
    - gainF (np.ndarray): Gain factors, shape (n_samples,)
    - risk (np.ndarray): True risks, shape (n_samples,)
    - cost (np.ndarray): Costs, shape (n_samples,)
    - race (np.ndarray): Race indicators (e.g., 0 for White, 1 for Black), shape (n_samples,)
    - Q (float): Total budget constraint
    - alphas (list of float or str): List of alpha fairness parameters

    Returns:
    - results_df (pd.DataFrame): Aggregated results comparing solver and closed-form
    - bias_analysis_df (pd.DataFrame): Bias analysis per race for both methods
    - solutions (dict): Contains optimal solutions from solver and closed-form
    """
    # Feature scaling
    scaler = StandardScaler()
    feats_scaled = scaler.fit_transform(feats)

    # Predict risks using the trained models
    model.eval()
    fair_model.eval()
    with torch.no_grad():
        pred_risk = model(torch.FloatTensor(feats_scaled)).numpy().flatten()
        fair_pred_risk = fair_model(torch.FloatTensor(feats_scaled)).numpy().flatten()

        pred_risk = pred_risk.clip(min=0.001)  # Ensure no zero values
        fair_pred_risk = fair_pred_risk.clip(min=0.001)  # Ensure no zero values

    risk = risk.clip(min=0.001)  # Ensure no zero values

    # Initialize result storage
    results = []
    bias_analysis = []
    solutions = {
        'solver': {},
        'closed_form': {}
    }

    # Iterate over alphas
    for alpha in alphas:
        # Solver-based solutions
        true_sol_solver, true_obj_solver = solve_optimization(gainF, risk, cost, alpha, Q)
        pred_sol_solver, pred_obj_solver = solve_optimization(gainF, pred_risk, cost, alpha, Q)
        fair_pred_sol_solver, fair_pred_obj_solver = solve_optimization(gainF, fair_pred_risk, cost, alpha, Q)

        # Closed-form solutions
        try:
            true_sol_cf = compute_d_star_closed_form(cost, risk, gainF, Q, alpha)
            pred_sol_cf = compute_d_star_closed_form(cost, pred_risk, gainF, Q, alpha)
            fair_pred_sol_cf = compute_d_star_closed_form(cost, fair_pred_risk, gainF, Q, alpha)
        except ValueError as e:
            print(f"Closed-form computation skipped for alpha={alpha}: {e}")
            true_sol_cf = pred_sol_cf = fair_pred_sol_cf = np.zeros_like(risk)

        # Store solutions
        solutions['solver'][alpha] = {
            'true_sol': true_sol_solver,
            'pred_sol': pred_sol_solver,
            'fair_pred_sol': fair_pred_sol_solver
        }
        solutions['closed_form'][alpha] = {
            'true_sol': true_sol_cf,
            'pred_sol': pred_sol_cf,
            'fair_pred_sol': fair_pred_sol_cf
        }

        # Compute utilities
        utilities_solver = gainF * risk * true_sol_solver
        pred_util_solver = gainF * risk * pred_sol_solver
        fair_pred_util_solver = gainF * risk * fair_pred_sol_solver

        utilities_cf = gainF * risk * true_sol_cf
        pred_util_cf = gainF * risk * pred_sol_cf
        fair_pred_util_cf = gainF * risk * fair_pred_sol_cf

        # Compute objectives using helper function
        true_obj_cf = AlphaFairness(utilities_cf, alpha)
        pred_obj_cf = AlphaFairness(pred_util_cf, alpha)
        fair_pred_obj_cf = AlphaFairness(fair_pred_util_cf, alpha)

        # Compute regrets and normalized regrets
        # Solver-based
        regret_solver = true_obj_solver - AlphaFairness(gainF * risk * pred_sol_solver, alpha)
        normalized_regret_solver = regret_solver / (abs(true_obj_solver) + 1e-7)

        fair_regret_solver = true_obj_solver - AlphaFairness(gainF * risk * fair_pred_sol_solver, alpha)
        fair_normalized_regret_solver = fair_regret_solver / (abs(true_obj_solver) + 1e-7)

        # Closed-form
        regret_cf = true_obj_cf - pred_obj_cf
        normalized_regret_cf = regret_cf / (abs(true_obj_cf) + 1e-7)

        fair_regret_cf = true_obj_cf - fair_pred_obj_cf
        fair_normalized_regret_cf = fair_regret_cf / (abs(true_obj_cf) + 1e-7)

        # Append solver results
        results.append({
            'Alpha': alpha,
            'Method': 'Solver',
            'Predicted Risk Mean': pred_risk.mean(),
            'True Risk Mean': risk.mean(),
            'True Objective': true_obj_solver,
            'Predicted Objective': AlphaFairness(gainF * risk * pred_sol_solver, alpha),
            'Regret': regret_solver,
            'Normalized Regret': normalized_regret_solver
        })

        # Append closed-form results
        results.append({
            'Alpha': alpha,
            'Method': 'Closed-Form',
            'Predicted Risk Mean': pred_risk.mean(),
            'True Risk Mean': risk.mean(),
            'True Objective': true_obj_cf,
            'Predicted Objective': pred_obj_cf,
            'Regret': regret_cf,
            'Normalized Regret': normalized_regret_cf
        })

        # Append fair solver results
        results.append({
            'Alpha': alpha,
            'Method': 'Solver (Fair)',
            'Predicted Risk Mean': fair_pred_risk.mean(),
            'True Risk Mean': risk.mean(),
            'True Objective': true_obj_solver,
            'Predicted Objective': AlphaFairness(gainF * risk * fair_pred_sol_solver, alpha),
            'Regret': fair_regret_solver,
            'Normalized Regret': fair_normalized_regret_solver
        })

        # Append fair closed-form results
        results.append({
            'Alpha': alpha,
            'Method': 'Closed-Form (Fair)',
            'Predicted Risk Mean': fair_pred_risk.mean(),
            'True Risk Mean': risk.mean(),
            'True Objective': true_obj_cf,
            'Predicted Objective': fair_pred_obj_cf,
            'Regret': fair_regret_cf,
            'Normalized Regret': fair_normalized_regret_cf
        })

        # Bias analysis for solver
        bias_solver = analyze_race_stats(
            alpha, race, true_sol_solver, pred_sol_solver, 
            utilities_solver, pred_util_solver
        )
        bias_analysis.extend([{'Method': 'Solver'} | stat for stat in bias_solver])

        # Bias analysis for closed-form
        bias_cf = analyze_race_stats(
            alpha, race, true_sol_cf, pred_sol_cf, 
            utilities_cf, pred_util_cf
        )
        bias_analysis.extend([{'Method': 'Closed-Form'} | stat for stat in bias_cf])

        # Bias analysis for fair solver
        bias_fair_solver = analyze_race_stats(
            alpha, race, true_sol_solver, fair_pred_sol_solver, 
            utilities_solver, fair_pred_util_solver
        )
        bias_analysis.extend([{'Method': 'Solver (Fair)'} | stat for stat in bias_fair_solver])

        # Bias analysis for fair closed-form
        bias_fair_cf = analyze_race_stats(
            alpha, race, true_sol_cf, fair_pred_sol_cf, 
            utilities_cf, fair_pred_util_cf
        )
        bias_analysis.extend([{'Method': 'Closed-Form (Fair)'} | stat for stat in bias_fair_cf])

    # Create DataFrames for results and bias analysis
    results_df = pd.DataFrame(results)
    bias_analysis_df = pd.DataFrame(bias_analysis)
    bias_analysis_df['Race'] = bias_analysis_df['Race'].astype(str)

    return results_df, bias_analysis_df, solutions

In [210]:
if __name__ == "__main__":
    # Define alpha values, including special cases
    alphas = [0.0, 0.5, 1.0, 2.0, 'inf']
    # model = RiskPredictor(feats.shape[1])
    # model.load_state_dict(torch.load('risk_predictor_model.pth'))

    # fair_model = FairRiskPredictor(feats.shape[1])
    # fair_model.load_state_dict(torch.load('fair_risk_predictor_model.pth'))
    
    # Run the analysis
    results_df, bias_analysis_df, solutions = twoStagePTO_with_bias_analysis(
        model, fair_model, feats_sample, gainF_sample, risk_sample, cost_sample, race_sample, Q=1000, alphas=alphas
    )

    # Display the aggregated results
    print("Aggregated Results:")
    print(results_df)

    # Display the bias analysis
    print("\nBias Analysis:")
    print(bias_analysis_df)


Aggregated Results:
   Alpha              Method  Predicted Risk Mean  True Risk Mean  \
0    0.0              Solver             2.703654        4.387572   
1    0.0         Closed-Form             2.703654        4.387572   
2    0.0       Solver (Fair)             2.149763        4.387572   
3    0.0  Closed-Form (Fair)             2.149763        4.387572   
4    0.5              Solver             2.703654        4.387572   
5    0.5         Closed-Form             2.703654        4.387572   
6    0.5       Solver (Fair)             2.149763        4.387572   
7    0.5  Closed-Form (Fair)             2.149763        4.387572   
8    1.0              Solver             2.703654        4.387572   
9    1.0         Closed-Form             2.703654        4.387572   
10   1.0       Solver (Fair)             2.149763        4.387572   
11   1.0  Closed-Form (Fair)             2.149763        4.387572   
12   2.0              Solver             2.703654        4.387572   
13   2.0      

In [227]:
results_df.groupby(['Alpha','Method']).mean()

Predicted Risk Mean  True Risk Mean  True Objective  \
Alpha Method                                                                    
0.0   Closed-Form                    2.703654        4.387572   283123.180400   
      Closed-Form (Fair)             2.149763        4.387572   283123.180400   
      Solver                         2.703654        4.387572   284164.896752   
      Solver (Fair)                  2.149763        4.387572   284164.896752   
0.5   Closed-Form                    2.703654        4.387572    73560.208282   
      Closed-Form (Fair)             2.149763        4.387572    73560.208282   
      Solver                         2.703654        4.387572    73629.284004   
      Solver (Fair)                  2.149763        4.387572    73629.284004   
1.0   Closed-Form                    2.703654        4.387572   -44928.603979   
      Closed-Form (Fair)             2.149763        4.387572   -44928.603979   
      Solver                         2.703654        4.387572   -44790.736205   
      Solver (Fair)                  2.149763        4.387572   -44790.736205   
2.0   Closed-Form                    2.703654        4.387572  -835094.022604   
      Closed-Form (Fair)             2.149763        4.387572  -835094.022604   
      Solver                         2.703654        4.387572  -587506.397186   
      Solver (Fair)                  2.149763        4.387572  -587506.397186   
inf   Closed-Form                    2.703654        4.387572        0.000627   
      Closed-Form (Fair)             2.149763        4.387572        0.000627   
      Solver                         2.703654        4.387572        0.001249   
      Solver (Fair)                  2.149763        4.387572        0.001249   

                          Predicted Objective        Regret  Normalized Regret  
Alpha Method                                                                    
0.0   Closed-Form                1.694669e+05  1.136563e+05           0.401438  
      Closed-Form (Fair)         1.668212e+05  1.163020e+05           0.410782  
      Solver                     1.694669e+05  1.146980e+05           0.403632  
      Solver (Fair)              1.668212e+05  1.173437e+05           0.412942  
0.5   Closed-Form                4.922819e+04  2.433202e+04           0.330777  
      Closed-Form (Fair)         4.501799e+04  2.854222e+04           0.388012  
      Solver                     4.967441e+04  2.395488e+04           0.325344  
      Solver (Fair)              4.550411e+04  2.812517e+04           0.381984  
1.0   Closed-Form               -4.492860e+04  0.000000e+00           0.000000  
      Closed-Form (Fair)        -4.492860e+04  0.000000e+00           0.000000  
      Solver                    -4.492860e+04  1.378681e+02           0.003078  
      Solver (Fair)             -4.492860e+04  1.378679e+02           0.003078  
2.0   Closed-Form               -3.661979e+06  2.826885e+06           3.385110  
      Closed-Form (Fair)        -5.609882e+06  4.774788e+06           5.717665  
      Solver                    -3.672405e+06  3.084899e+06           5.250835  
      Solver (Fair)             -5.621857e+06  5.034351e+06           8.569015  
inf   Closed-Form                2.124773e-06  6.246730e-04           0.996451  
      Closed-Form (Fair)         7.966767e-07  6.260011e-04           0.998570  
      Solver                     2.540136e-06  1.246820e-03           0.997887  
      Solver (Fair)              9.241947e-07  1.248436e-03           0.999180

: 

In [224]:
bias_analysis_df.groupby(['Method', 'Alpha','Race']).mean(numeric_only=True).round(4)

True Solution Mean  True Solution Std  \
Method             Alpha Race                                           
Closed-Form        0.0   Black              0.0000             0.0000   
                         White              0.0282             5.3127   
                   0.5   Black              0.0232             0.0359   
                         White              0.0252             0.0362   
                   1.0   Black              0.0250             0.0000   
                         White              0.0250             0.0000   
                   2.0   Black              0.0269             0.2277   
                         White              0.0248             0.2157   
                   inf   Black              0.0275             0.4137   
                         White              0.0247             0.3918   
Closed-Form (Fair) 0.0   Black              0.0000             0.0000   
                         White              0.0282             5.3127   
                   0.5   Black              0.0232             0.0359   
                         White              0.0252             0.0362   
                   1.0   Black              0.0250             0.0000   
                         White              0.0250             0.0000   
                   2.0   Black              0.0269             0.2277   
                         White              0.0248             0.2157   
                   inf   Black              0.0275             0.4137   
                         White              0.0247             0.3918   
Solver             0.0   Black              0.0000             0.0000   
                         White              0.0282             5.3127   
                   0.5   Black              0.0232             0.0360   
                         White              0.0252             0.0363   
                   1.0   Black              0.0250             0.0000   
                         White              0.0250             0.0000   
                   2.0   Black              0.0268             0.1917   
                         White              0.0248             0.1816   
                   inf   Black              0.0275             0.4123   
                         White              0.0247             0.3905   
Solver (Fair)      0.0   Black              0.0000             0.0000   
                         White              0.0282             5.3127   
                   0.5   Black              0.0232             0.0360   
                         White              0.0252             0.0363   
                   1.0   Black              0.0250             0.0000   
                         White              0.0250             0.0000   
                   2.0   Black              0.0268             0.1917   
                         White              0.0248             0.1816   
                   inf   Black              0.0275             0.4123   
                         White              0.0247             0.3905   

                                Predicted Solution Mean  \
Method             Alpha Race                             
Closed-Form        0.0   Black                   0.0000   
                         White                   0.0282   
                   0.5   Black                   0.0288   
                         White                   0.0245   
                   1.0   Black                   0.0250   
                         White                   0.0250   
                   2.0   Black                   0.0223   
                         White                   0.0253   
                   inf   Black                   0.0261   
                         White                   0.0249   
Closed-Form (Fair) 0.0   Black                   0.0000   
                         White                   0.0282   
                   0.5   Black                   0.0309   
                         White                   0.0242   
                   1.